In [2]:
### required libraries for gathering and cleaning the data
import numpy as np
import pandas as pd
import requests
import tweepy
import json
import io

#  GATHERING DATA

Gathering Data for this Project
Gather each of the three pieces of data as described below in a Jupyter Notebook titled wrangle_act.ipynb:

1. The WeRateDogs Twitter archive via manual download
2. Programatically for Tweet image predictions via https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv
3. Query the Twitter API via IDS from weratedogs twitter archive.

## The WeRateDogs Twitter archive

The WeRateDogs Twitter archive. I am giving this file to you, so imagine it as a file on hand. Download this file manually by clicking the following link: twitter_archive_enhanced.csv

In [27]:
twitter_arc = pd.read_csv("twitter-archive-enhanced.csv")
twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2176,669203728096960512,NaN,NaN,2015-11-24 17:19:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Samsom. He is sexually confused. Reall...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669203728...,9,10,Samsom,None,None,None,None
1858,675501075957489664,NaN,NaN,2015-12-12 02:23:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I shall call him squishy and he shall be mine,...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/675501075...,13,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,13615722.0,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None
512,811744202451197953,NaN,NaN,2016-12-22 01:24:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Halo. She likes watermelon. 13/10 http...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/811744202...,13,10,Halo,None,None,None,None
128,867774946302451713,NaN,NaN,2017-05-25 16:10:44 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Harold. He's h*ckin cooperative. 13/10 g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/867774946...,13,10,Harold,None,None,None,None


In [28]:
twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

## Tweet image predictions

The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

In [31]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

In [32]:
requests.get(url).encoding

'utf-8'

In [33]:
r = requests.get(url).content

In [34]:
tweet_image = pd.read_table(io.StringIO(r.decode('utf-8'))) # io.StringIO is for reading a table from a string

In [35]:
tweet_image.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
67,667177989038297088,https://pbs.twimg.com/media/CUJK18UWEAEg7AR.jpg,1,vizsla,0.259249,True,Chesapeake_Bay_retriever,0.176293,True,Weimaraner,0.112369,True
1653,809920764300447744,https://pbs.twimg.com/media/Cz1qo05XUAQ4qXp.jpg,1,Norwich_terrier,0.397163,True,toy_poodle,0.274540,True,miniature_poodle,0.134667,True
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
1720,819347104292290561,https://pbs.twimg.com/media/C17n1nrWQAIErU3.jpg,3,Rottweiler,0.909106,True,black-and-tan_coonhound,0.044120,True,Doberman,0.031835,True
1894,849776966551130114,https://pbs.twimg.com/media/C8sDpDWWsAE5P08.jpg,2,Chihuahua,0.292092,True,toy_terrier,0.136852,True,bonnet,0.103111,False
143,668620235289837568,https://pbs.twimg.com/media/CUdqjvAWUAANfoU.jpg,1,crash_helmet,0.757942,False,toaster,0.037497,False,mouse,0.027271,False
1552,793150605191548928,https://pbs.twimg.com/media/CwHWOZ7W8AAHv8S.jpg,1,Italian_greyhound,0.193869,True,bluetick,0.160380,True,standard_poodle,0.125982,True
1566,793962221541933056,https://pbs.twimg.com/media/CwS4aqZXUAAe3IO.jpg,1,Labrador_retriever,0.861651,True,golden_retriever,0.044462,True,Staffordshire_bullterrier,0.016497,True
1151,732375214819057664,https://pbs.twimg.com/media/CinrX2EWkAABDYt.jpg,1,tennis_ball,0.998673,False,basset,0.000547,True,golden_retriever,0.000360,True
1264,749064354620928000,https://pbs.twimg.com/media/CmU2DVWWgAArvp3.jpg,2,pug,0.985222,True,Brabancon_griffon,0.003314,True,Pekinese,0.002989,True


In [36]:
tweet_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


## Query the Twitter API 

Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. Note: do not include your Twitter API keys, secrets, and tokens in your project submission.

## DELETE the actual key, secret, and token but leave for referance

In [4]:
import tweepy

consumer_key = 'd61pDmkhtuSYm6nOXGPIgjkpf'
consumer_secret = 'SpRV5uDJyfNbBjsbLHR6BoSdoenPNydhHBpYIIHsQPtORuPRSX'

access_token = '991423096849510400-HyoD9ZT2LKNOJVCu9eVTccIQrvSK7uf'
access_secret = 'LCWFkbuTJcwwiDziCcypbKRXs9OAgozDGumLGmWdWj1hk'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) ### http://docs.tweepy.org/en/v3.2.0/api.html#API

In [5]:
# list of ids from WeRateDogs Twitter's archice for querying
# sum(twiter_arc_enc["tweet_id"].duplicated())
query_list = list(twiter_arc_enc["tweet_id"])

In [6]:
len(query_list) / 4 ### split querying the twitter API in 4 groups 0:589, 590:1179, 1180:1769, 1770:2356

589.0

http://python.mfamt.org/content/recipes/apis-and-scraping/tweepy-introduction/

In [17]:
### I have commented this section out so I accidentaly don t run it again :)
list_of_tweetsJSONs = []
for i in query_list:
    try:
        tweet = api.get_status(i, tweet_mode='extended')
        list_of_tweetsJSONs.append(tweet._json)
        ## This cell is slow so print ranking to gauge time remaining
        ranking = query_list.index(i) + 1
        print(ranking)
    except Exception as e:
        print(str(e))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
[{'code': 144, 'message': 'No status found with that ID.'}]
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
[{'code': 144, 'message': 'No status found with that ID.'}]
97
98
99
100
101
[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
[{'code': 144, 'message': 'No status found with that ID.'}]
120
121
122
123
124
125
126
127
128
129
130
131
132
[{'code': 144, 'message': 'No status found with that ID.'}]
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
[{'code': 144, 'message': 'No status found with that ID.'}]
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192


1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776


In [20]:
### 18 of the IDs are not found on the twitter : some due to network issues
len(query_list)-len(list_of_tweetsJSONs)

18

In [19]:
### write each json element from a list of tweets in a txt file as a new line

#filename = 'tweet_json.txt'

#with open(filename, mode="w") as file:  # also, tried mode="rb"
#    for d in list_of_tweetsJSONs:
#        file.write(json.dumps(d)  + "\n")

In [21]:
### read tweet_json.txt and create pandas data frame for selected json columns
tweet_id = []
retweeted = []
favourites_count = []
retweet_count = []
created_at = []

with open('tweet_json.txt') as json_file:
    for line in json_file.readlines():
        data = json.loads(line)
        tweet_id.append(data["id"])
        retweeted.append(data["retweeted"]) 
        favourites_count.append(data["user"]["favourites_count"])
        retweet_count.append(data["retweet_count"])
        created_at.append(data["created_at"])


In [22]:
### build pandas dataframe from lists
twitter_api = pd.DataFrame({"tweet_id" : tweet_id, "retweeted" : retweeted, "retweet_count": retweet_count,"favourites_count" : favourites_count,  "created_at" : created_at})

In [23]:
twitter_api.head()

,tweet_id,retweeted,retweet_count,favourites_count,created_at
0,892420643555336193,False,8426,138439,Tue Aug 01 16:23:56 +0000 2017
1,892177421306343426,False,6208,138439,Tue Aug 01 00:17:27 +0000 2017
2,891815181378084864,False,4107,138439,Mon Jul 31 00:18:03 +0000 2017
3,891689557279858688,False,8540,138439,Sun Jul 30 15:58:51 +0000 2017
4,891327558926688256,False,9279,138439,Sat Jul 29 16:00:24 +0000 2017


### Dataframes for assesment

In [29]:
twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
67,879376492567855104,NaN,NaN,2017-06-26 16:31:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jack AKA Stephen Furry. You're not sco...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/879376492...,12,10,Jack,None,None,None,None
608,797971864723324932,NaN,NaN,2016-11-14 01:18:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a helicopter pupper. He takes off at ra...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/797971864...,12,10,None,None,None,pupper,None
2013,672231046314901505,NaN,NaN,2015-12-03 01:49:05 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Exotic underwater dog here. Very shy. Wont ret...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672231046...,5,10,None,None,None,None,None
146,863471782782697472,NaN,NaN,2017-05-13 19:11:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Say hello to Quinn. She's quite...,8.630625e+17,4.196984e+09,2017-05-12 16:05:02 +0000,"https://www.gofundme.com/helpquinny,https://tw...",13,10,Quinn,None,None,None,None
1358,703382836347330562,NaN,NaN,2016-02-27 00:55:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Charlie. He's a West Side Niddlewog. M...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/703382836...,12,10,Charlie,None,None,None,None


In [37]:
tweet_image.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1088,718971898235854848,https://pbs.twimg.com/media/CfpNGTHUIAAA8XC.jpg,1,golden_retriever,0.140394,True,Saint_Bernard,0.118769,True,Labrador_retriever,0.075492,True
547,677331501395156992,https://pbs.twimg.com/media/CWZdaGxXAAAjGjb.jpg,1,beagle,0.313464,True,boxer,0.218503,True,French_bulldog,0.106462,True
166,668981893510119424,https://pbs.twimg.com/media/CUize-0WEAAerAK.jpg,1,jellyfish,0.447246,False,coral_reef,0.238625,False,goldfish,0.040227,False
478,675334060156301312,https://pbs.twimg.com/media/CV9EvZNUwAAgLCK.jpg,2,Pembroke,0.773135,True,Cardigan,0.116810,True,chow,0.039036,True
1694,816091915477250048,https://pbs.twimg.com/media/C1NXQ6NXUAEAxIQ.jpg,3,Pomeranian,0.967345,True,Samoyed,0.007397,True,papillon,0.006017,True


In [38]:
twitter_api.sample(5)

,tweet_id,retweeted,retweet_count,favourites_count,created_at
159,860177593139703809,False,31743,138439,Thu May 04 17:01:34 +0000 2017
78,877556246731214848,False,3782,138439,Wed Jun 21 15:58:08 +0000 2017
914,754120377874386944,False,2548,138438,Sat Jul 16 01:08:03 +0000 2016
2036,671390180817915904,False,762,138442,Mon Nov 30 18:07:47 +0000 2015
1166,718631497683582976,False,8677,138444,Sat Apr 09 02:47:55 +0000 2016


# Assessment

Assessing Data for this Project
After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues. Detect and document at least eight (8) quality issues and two (2) tidiness issues in your wrangle_act.ipynb Jupyter Notebook. To meet specifications, the issues that satisfy the Project Motivation (see the Key Points header on the previous page) must be assessed.

You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

Visual assessment: each piece of gathered data is displayed in the Jupyter Notebook for visual assessment purposes. Once displayed, data can additionally be assessed in an external application (e.g. Excel, text editor).

Programmatic assessment: pandas' functions and/or methods are used to assess the data.

### Visual assessment

In [41]:
twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
385,826848821049180160,NaN,NaN,2017-02-01 17:44:55 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cupid. He was found in the trash. Now ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/826848821...,13,10,Cupid,doggo,None,None,None
1959,673576835670777856,NaN,NaN,2015-12-06 18:56:46 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Sun burnt dog here. Quite large. Wants to prom...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673576835...,7,10,None,None,None,None,None
1545,689289219123089408,NaN,NaN,2016-01-19 03:32:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Someone sent me this without any context and e...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/689289219...,13,10,None,None,None,None,None
1941,673715861853720576,NaN,NaN,2015-12-07 04:09:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a heavily opinionated dog. Loves walls...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673715861...,4,10,a,None,None,None,None
1325,706166467411222528,NaN,NaN,2016-03-05 17:16:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dexter. He's a shy pup. Doesn't bark m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/706166467...,7,10,Dexter,None,None,None,None


In [42]:
tweet_image.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1743,822859134160621569,https://pbs.twimg.com/media/C2tiAzGXgAIFdqi.jpg,1,malinois,0.332897,True,Chihuahua,0.104116,True,Staffordshire_bullterrier,0.047745,True
2052,887517139158093824,https://pbs.twimg.com/ext_tw_video_thumb/88751...,1,limousine,0.130432,False,tow_truck,0.029175,False,shopping_cart,0.026321,False
279,671109016219725825,https://pbs.twimg.com/media/CVBCFkyU4AE2Wcr.jpg,1,basenji,0.855959,True,beagle,0.036723,True,toy_terrier,0.029258,True
1366,761672994376806400,https://pbs.twimg.com/ext_tw_video_thumb/76167...,1,gondola,0.318851,False,sea_lion,0.306525,False,pool_table,0.111565,False
1937,860276583193509888,https://pbs.twimg.com/media/C_BQ_NlVwAAgYGD.jpg,1,lakeside,0.312299,False,dock,0.159842,False,canoe,0.070795,False


In [43]:
twitter_api.sample(5)

,tweet_id,retweeted,retweet_count,favourites_count,created_at
2203,668480044826800133,False,153,138444,Sun Nov 22 17:23:57 +0000 2015
174,857062103051644929,False,177,138439,Wed Apr 26 02:41:43 +0000 2017
766,775898661951791106,False,17323,138439,Wed Sep 14 03:27:11 +0000 2016
1209,713909862279876608,False,585,138444,Sun Mar 27 02:05:49 +0000 2016
756,777621514455814149,False,2750,138439,Sun Sep 18 21:33:11 +0000 2016


In [44]:
pd.options.display.max_colwidth = 150
pd.set_option('display.max_rows', 2000)

In [46]:
### tweet_id 883482846933004288 has a decimal 13.5; 667550882905632768 has -5 
twitter_arc.query("tweet_id == 883482846933004288") 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
45,883482846933004288,NaN,NaN,2017-07-08 00:28:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",5,10,Bella,None,None,None,None


In [47]:
twitter_arc[["tweet_id","text", "rating_numerator"]]

,tweet_id,text,rating_numerator
0,892420643555336193,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,13
1,892177421306343426,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https:/...",13
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,12
3,891689557279858688,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,13
4,891327558926688256,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https:/...",12
5,891087950875897856,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https:/...,13
6,890971913173991426,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H htt...,13
7,890729181411237888,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,13
8,890609185150312448,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,13
9,890240255349198849,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co...,14


### Tidiness

Display random observations from all 3 tables to visually inspect for tidines issues:

**twitter_arc dataframe:**
 
1. None values for doggo, floofer, pupper, puppo columns, since there is only one value from four for every row filled, it means that data needs to be melted in a column dog_stage **(done)**

**tweet_image dataframe:**

2. dataframe can be joined onto twiter_arc **(done)**


**twitter_api dataframe:**

1. dataframe can be joined to twiter_arc **(done)**

### Quality 

**twiter_arc dataframe:**

1. rating_numerator does not have correct numbers extracted from status tweets for example ### tweet_id 883482846933004288 has a decimal 13.5

2. Columns not required - in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp

3. Source column contains html links embeded in html as strings **(done)**



**tweet_image dataframe:**

1. p1. p2 and p3 columns have a lot of variations in the cells, for example: - or _ as a space between words, varied capitalsiation - capitalize and remove - and _   **(done)**

2. remove p1_dog, p2_dog and p3_dog == False ("Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.") **(done)**

**twitter_api dataframe:**



### Programatical assessment

### Tidiness and Quality

In [50]:
twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [51]:
tweet_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [52]:
twitter_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 5 columns):
tweet_id            2338 non-null int64
retweeted           2338 non-null bool
retweet_count       2338 non-null int64
favourites_count    2338 non-null int64
created_at          2338 non-null object
dtypes: bool(1), int64(3), object(1)
memory usage: 75.4+ KB


In [53]:
twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [54]:
twitter_arc.describe(include=['O'])

,timestamp,source,text,retweeted_status_timestamp,expanded_urls,name,doggo,floofer,pupper,puppo
count,2356,2356,2356,181,2297,2356,2356,2356,2356,2356
unique,2356,4,2356,181,2218,957,2,2,2,2
top,2016-11-08 22:25:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Loki. He knows he's adorable. One ear always pupared. 12/10 would snug in depicted fashion forever https://t.co/OqNggd4Oio,2016-12-12 16:16:49 +0000,"https://www.gofundme.com/3yd6y1c,https://twitter.com/dog_rates/status/878281511006478336/photo/1",None,None,None,None,None
freq,1,2221,1,1,2,745,2259,2346,2099,2326


In [55]:
twitter_arc.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [56]:
sum(twitter_arc["tweet_id"].duplicated())

0

In [57]:
sum(twitter_arc["retweeted_status_user_id"].duplicated())

2330

In [58]:
tweet_image.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [59]:
tweet_image.describe(include=['O'])

,jpg_url,p1,p2,p3
count,2075,2075,2075,2075
unique,2009,378,405,408
top,https://pbs.twimg.com/media/DA7iHL5U0AA1OQo.jpg,golden_retriever,Labrador_retriever,Labrador_retriever
freq,2,150,104,79


In [60]:
sum(tweet_image["tweet_id"].duplicated())

0

In [61]:
twitter_api.describe()

,tweet_id,retweet_count,favourites_count
count,2.338000e+03,2338.000000,2338.000000
mean,7.421499e+17,2966.723695,138441.633875
std,6.834493e+16,4982.374820,2.411337
min,6.660209e+17,0.000000,138438.000000
25%,6.783361e+17,596.250000,138439.000000
50%,7.185003e+17,1384.000000,138443.000000
75%,7.986971e+17,3458.750000,138444.000000
max,8.924206e+17,84506.000000,138444.000000


In [62]:
sum(twitter_api["retweet_count"] == 0)

1

In [63]:
twitter_api.describe(include=['O'])

,created_at
count,2338
unique,2338
top,Thu Mar 31 23:22:53 +0000 2016
freq,1


In [64]:
sum(twitter_api["tweet_id"].duplicated())

0

In [65]:
twitter_arc["rating_denominator"].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [66]:
twitter_arc["rating_numerator"].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

### Tidiness

All tidiness questions have been addressed in visual inspection.

### Quality 

** twitter_arc** dataframe:

1. There are 6 columns that have non-null objects.

2. Timestamp to be converted to time object **(done)**

3. rating_denumerator and rating_numerator have max values of 1776 and 170 - investigate since 75th percentile is 12 and 10 respectively.  **(will not fix since ratings are fictional)**

4. rating_denominator should be 10, so all other values to be converted to 10.  **(done)**

**tweet_image** dataframe:

1. name column has strings such as "a", "an", "his", "my", "one"  **(done)**

**twitter_api** dataframe:

1. created_at should be converted from object to timestamp **(done)**


* in all tables, tweet_id shoud be a string and not int **(done)**


# Cleaning

Create copies of dataframes for cleaning:

In [68]:
twitter_arc_copy = twitter_arc.copy()
tweet_image_copy = tweet_image.copy()
twitter_api_copy = twitter_api.copy()

Remove retweets and keep only those rows that have images ("You only want original ratings (no retweets) that have images")

In [69]:
twitter_arc_copy.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [70]:
tweet_image_copy.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

In [71]:
# keep only rows where retweeted_status_user_id is NaN
twitter_arc_copy = twitter_arc_copy[pd.isnull(twitter_arc_copy['retweeted_status_user_id'])] 

Before joins, convert tweet_id to strings

In [72]:
twitter_arc_copy["tweet_id"] =  pd.Series.astype(twitter_arc_copy["tweet_id"],  dtype='object')
tweet_image_copy["tweet_id"] =  pd.Series.astype(tweet_image_copy["tweet_id"],dtype='object') 
twitter_api_copy["tweet_id"] =  pd.Series.astype(twitter_api_copy["tweet_id"],dtype='object')

In [73]:
twitter_api_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 5 columns):
tweet_id            2338 non-null object
retweeted           2338 non-null bool
retweet_count       2338 non-null int64
favourites_count    2338 non-null int64
created_at          2338 non-null object
dtypes: bool(1), int64(2), object(2)
memory usage: 75.4+ KB


In [75]:
# merge twiter_arc_copy and tweet_image_copy on twitter_id
merged_2 = pd.merge(twitter_arc_copy, tweet_image_copy, how='left', on="tweet_id")
# join merged and twitter_api_copy on twitter_id
merged_3 = pd.merge(merged_2, twitter_api_copy, how='left', on="tweet_id")

In [77]:
# remove rows where jpg_url is NaN
merged_3 = merged_3[pd.notnull(merged_3['jpg_url'])] 

In [78]:
merged_copy = merged_3.copy()

In [79]:
merged_copy.shape

(1994, 32)

In [80]:
merged_copy.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo',
       'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf',
       'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'retweeted', 'retweet_count',
       'favourites_count', 'created_at'],
      dtype='object')

In [81]:
# remove uneccessary columns
merged_copy = merged_copy.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis=1)

#### DEFINE

Improve extraction of the rating_numerator (examples of values not extracted:tweet_id 883482846933004288 has a decimal 13.5). Use regular expressions to capture decimal numbers.

#### CODE

In [83]:
merged_copy.query("tweet_id == 883482846933004288")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweeted,retweet_count,favourites_count,created_at
42,883482846933004288,NaN,NaN,2017-07-08 00:28:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948","https://twitter.com/dog_rates/status/883482846933004288/photo/1,https://twitter.com/dog_rates/status/883482846933004288/photo/1",5,10,Bella,...,Labrador_retriever,0.032409,True,kuvasz,0.005501,True,False,9869.0,138439.0,Sat Jul 08 00:28:19 +0000 2017


In [84]:
merged_copy["rating_numerator"] = merged_copy["text"].str.extract(r"(-?\d*[.]?\d+)/")

#### TEST

In [86]:
merged_copy[["text","rating_numerator"]]

,text,rating_numerator
0,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,13
1,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https:/...",13
2,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,12
3,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,13
4,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https:/...",12
5,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https:/...,13
6,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H htt...,13
7,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,13
8,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,13
9,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co...,14


####  DEFINE

Convert doggo, floofer, pupper, puppo None to NaN's. Create a flag for a dog stage and use the flag to create a new column called dog_stage.

#### CODE

In [92]:
# replace "None" with NaN
merged_copy = merged_copy.replace("None",np.nan)
merged_copy[['tweet_id', 'doggo', 'floofer', 'pupper', 'puppo']]

,tweet_id,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,NaN,NaN
5,891087950875897856,NaN,NaN,NaN,NaN
6,890971913173991426,NaN,NaN,NaN,NaN
7,890729181411237888,NaN,NaN,NaN,NaN
8,890609185150312448,NaN,NaN,NaN,NaN
9,890240255349198849,doggo,NaN,NaN,NaN


In [94]:
# concatenate 4 columns in one and fill NaN with empty string
merged_copy["dog_stage"] = merged_copy[['doggo', 'floofer', 'pupper', 'puppo']].fillna('').sum(axis=1)

In [95]:
# replace "" with NaN
merged_copy = merged_copy.replace("",np.nan)

In [96]:
# drop uneccessary columns
merged_copy = merged_copy.drop(['doggo', 'floofer', 'pupper', 'puppo'
                               ], axis=1)

#### TEST

In [100]:
merged_copy.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweeted,retweet_count,favourites_count,created_at,dog_stage
587,782598640137187329,NaN,NaN,2016-10-02 15:10:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Timmy. He's quite large. According to a trusted source it's actually a dog wearing a dog suit. 11/10 https://t.co/BIUchFwHqn,https://twitter.com/dog_rates/status/782598640137187329/photo/1,11,10,Timmy,...,0.140516,True,Eskimo_dog,0.012012,True,False,2078.0,138439.0,Sun Oct 02 15:10:30 +0000 2016,NaN
155,857263160327368704,NaN,NaN,2017-04-26 16:00:39 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Oscar and Oliver. Oliver shrunk Oscar. Oscar isn't pleased about it. Quite pupset tbh. Oliver doesn't seem to mind. Both 13/10 https://t.c...,https://twitter.com/dog_rates/status/857263160327368704/photo/1,13,10,Oscar,...,0.000922,True,keeshond,0.000311,True,False,4687.0,138439.0,Wed Apr 26 16:00:39 +0000 2017,NaN
1071,711008018775851008,NaN,NaN,2016-03-19 01:54:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Chuckles. He had a balloon but he accidentally let go of it and it floated away. 11/10 hang in there pupper https://t.co/68iNM7B5gW,https://twitter.com/dog_rates/status/711008018775851008/photo/1,11,10,Chuckles,...,0.150672,True,pug,0.021811,True,False,667.0,138444.0,Sat Mar 19 01:54:56 +0000 2016,pupper


In [101]:
merged_copy["dog_stage"].value_counts() 
###note that doggopupper, doggopuppo and doggofloofer are a concatination since they have been present for the same tweet_id in two categories

pupper          203
doggo            63
puppo            22
doggopupper       9
floofer           7
doggofloofer      1
doggopuppo        1
Name: dog_stage, dtype: int64

In [102]:
merged_3.shape[0] == merged_copy.shape[0]

True

####  DEFINE

Name column is not always representing a name (a - 55, the - 7, an - 6, my - 1, one - 4, his - 1). Replace these values with NaN.

#### CODE

In [105]:
merged_copy["name"] = merged_copy["name"].replace("a",np.nan)
merged_copy["name"] = merged_copy["name"].replace("the",np.nan)
merged_copy["name"] = merged_copy["name"].replace("an",np.nan)
merged_copy["name"] = merged_copy["name"].replace("my",np.nan)
merged_copy["name"] = merged_copy["name"].replace("one",np.nan)
merged_copy["name"] = merged_copy["name"].replace("his",np.nan)

#### TEST

In [106]:
merged_copy.name.value_counts()

Charlie           11
Oliver            10
Cooper            10
Lucy              10
Penny              9
Tucker             9
Winston            8
Sadie              8
Daisy              7
Lola               7
Toby               7
Stanley            6
Jax                6
Bella              6
Bo                 6
Koda               6
Rusty              5
Dave               5
Milo               5
Scout              5
Leo                5
Louis              5
Buddy              5
Bailey             5
Oscar              5
Chester            5
Bear               4
Jerry              4
Alfie              4
Dexter             4
Duke               4
very               4
Larry              4
George             4
Winnie             4
Phil               4
Sophie             4
Brody              4
Bentley            4
Jack               4
Derek              4
Scooter            4
Maggie             4
Clark              4
Clarence           4
Gary               4
Finn               4
Reggie       

####  DEFINE

Parse time column (object) as timestamp.


#### CODE

In [107]:
merged_copy["timestamp"].head()

0    2017-08-01 16:23:56 +0000
1    2017-08-01 00:17:27 +0000
2    2017-07-31 00:18:03 +0000
3    2017-07-30 15:58:51 +0000
4    2017-07-29 16:00:24 +0000
Name: timestamp, dtype: object

In [108]:
## http://strftime.org/
merged_copy["timestamp"] =  pd.to_datetime(merged_copy["timestamp"]) 


#### TEST

In [109]:
merged_copy["timestamp"].head()

0   2017-08-01 16:23:56
1   2017-08-01 00:17:27
2   2017-07-31 00:18:03
3   2017-07-30 15:58:51
4   2017-07-29 16:00:24
Name: timestamp, dtype: datetime64[ns]

####  DEFINE

rating_denominator should be 10, so all other values are converted to 10.

#### CODE

In [110]:
# replace "rating_denominator > 10 with 10
merged_copy["rating_denominator"] = merged_copy.replace(merged_copy.query("rating_denominator > 10"), 10)

In [113]:
merged_copy["rating_denominator"] = np.where(merged_copy["rating_denominator"]>10, 10, merged_copy["rating_denominator"])

#### TEST

In [114]:
merged_copy.query("rating_denominator > 10").shape[0] == 0

True

####  DEFINE

Clean html tags from merged_copy["source"] strings.

#### CODE

In [116]:
merged_copy["source"] = merged_copy["source"].str.extract('(http://[a-zA-Z]*.com/[a-zA-Z]*)')

#### TEST

In [117]:
merged_copy["source"].sample(5)

1165    http://twitter.com/download
442     http://twitter.com/download
1001    http://twitter.com/download
2154    http://twitter.com/download
803                             NaN
Name: source, dtype: object

####  DEFINE

for p1, p2 and p3 columns - remove rows which are not dogs, i.e. dog flag == False, then drop the p1_dog, p2_dog, p3_dog columns

In [118]:
merged_copy.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'jpg_url', 'img_num', 'p1', 'p1_conf',
       'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog',
       'retweeted', 'retweet_count', 'favourites_count', 'created_at',
       'dog_stage'],
      dtype='object')

##### CODE

In [120]:
merged_copy = merged_copy.query("p1_dog == True") 
merged_copy = merged_copy.query("p2_dog == True") 
merged_copy = merged_copy.query("p3_dog == True") 

#### TEST

In [122]:
print(merged_copy.query("p1_dog == False").shape[0]), print(merged_copy.query("p2_dog == False").shape[0]), print(merged_copy.query("p3_dog == False").shape[0])

0
0
0


(None, None, None)

In [123]:
# remove columns for flags
merged_copy.drop(["p1_dog", "p2_dog", "p3_dog"], axis=1, inplace=True)
merged_copy.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'jpg_url', 'img_num', 'p1', 'p1_conf',
       'p2', 'p2_conf', 'p3', 'p3_conf', 'retweeted', 'retweet_count',
       'favourites_count', 'created_at', 'dog_stage'],
      dtype='object')

####  DEFINE

p1, p2 and p3 columns: lower case and remove "_" and "-"

#### CODE

In [124]:
merged_copy["p1"] = merged_copy["p1"].replace("[-_]", ' ', regex=True).str.lower()
merged_copy["p2"] = merged_copy["p2"].replace("[-_]", ' ', regex=True).str.lower()
merged_copy["p3"] = merged_copy["p3"].replace("[-_]", ' ', regex=True).str.lower()

#### TEST

In [125]:
merged_copy["p1"].sample(5)

1641           boston bull
641     norwegian elkhound
654               papillon
551                   chow
747               pembroke
Name: p1, dtype: object

####  DEFINE

Convert created_at as datetime

#### CODE

In [127]:
merged_copy["created_at"] =  pd.to_datetime(merged_copy["created_at"]) 

#### TEST

In [128]:
merged_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1203 entries, 1 to 2174
Data columns (total 23 columns):
tweet_id                 1203 non-null int64
in_reply_to_status_id    9 non-null float64
in_reply_to_user_id      9 non-null float64
timestamp                1203 non-null datetime64[ns]
source                   1184 non-null object
text                     1203 non-null object
expanded_urls            1203 non-null object
rating_numerator         1203 non-null object
rating_denominator       1203 non-null object
name                     865 non-null object
jpg_url                  1203 non-null object
img_num                  1203 non-null float64
p1                       1203 non-null object
p1_conf                  1203 non-null float64
p2                       1203 non-null object
p2_conf                  1203 non-null float64
p3                       1203 non-null object
p3_conf                  1203 non-null float64
retweeted                1201 non-null object
retweet_count

In [129]:
merged_copy.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,...,p1_conf,p2,p2_conf,p3,p3_conf,retweeted,retweet_count,favourites_count,created_at,dog_stage
161,856282028240666624,NaN,NaN,2017-04-23 23:01:59,http://twitter.com/download,"This is Cermet, Paesh, and Morple. They are absolute h*ckin superstars. Watered every day so they can grow. 14/10 for all https://t.co/GUefqUmZv8","https://twitter.com/dog_rates/status/856282028240666624/photo/1,https://twitter.com/dog_rates/status/856282028240666624/photo/1,https://twitter.co...",14,10,Cermet,...,0.876543,italian greyhound,0.032962,cardigan,0.020776,False,6565.0,138439.0,2017-04-23 23:01:59,NaN
595,781163403222056960,NaN,NaN,2016-09-28 16:07:23,http://twitter.com/download,"We normally don't rate lobsters, but this one appears to be a really good lobster. 10/10 would pet with caution https://t.co/YkHc7U7uUy",https://twitter.com/dog_rates/status/781163403222056960/photo/1,10,10,NaN,...,0.973841,collie,0.025188,border collie,0.000297,False,3013.0,138439.0,2016-09-28 16:07:23,NaN
2107,667176164155375616,NaN,NaN,2015-11-19 03:02:47,http://twitter.com/download,These are strange dogs. All have toupees. Long neck for dogs. In a shed of sorts? Work in groups? 4/10 still petable https://t.co/PZxSarAfSN,https://twitter.com/dog_rates/status/667176164155375616/photo/1,4,10,NaN,...,0.318981,lakeland terrier,0.215218,toy poodle,0.106014,False,471.0,138444.0,2015-11-19 03:02:47,NaN


## Export of wrangeled and clean table

In [130]:
merged_copy.to_csv("twitter_archive_master.csv", sep=',', encoding='utf-8')